# 3D Frozen-in Magnetic Flux (Rm >> 1) - Cartesian

**Validation case**: 3D Cartesian advection in the ideal MHD limit

## Learning Objectives

After completing this notebook, you will understand:

1. The magnetic diffusion equation and its ideal-MHD limit
2. Why magnetic flux is frozen to the plasma when $R_m \gg 1$
3. The Cartesian analytic solution for uniform $\mathbf{v}$ and $\mathbf{B}$
4. How to validate a solver against a constant-field frozen-flux case


---
## 1. Physics Background

### Magnetic Diffusion (Cartesian)

From the magnetic diffusion derivation, the induction equation is:

$$\frac{\partial \mathbf{B}}{\partial t} = \nabla \times (\mathbf{v} \times \mathbf{B}) + \frac{1}{\mu_0 \sigma} \nabla^2 \mathbf{B}$$

Define the magnetic Reynolds number $R_m = \mu_0 \sigma V L$. In the **ideal-MHD limit** ($R_m \gg 1$),

$$\frac{\partial \mathbf{B}}{\partial t} = \nabla \times (\mathbf{v} \times \mathbf{B}).$$

For **uniform** $\mathbf{v}$ and **uniform** $\mathbf{B}$ in Cartesian coordinates, $\nabla \times (\mathbf{v} \times \mathbf{B}) = 0$.
Therefore, the analytic solution is simply

$$\mathbf{B}(t) = \mathbf{B}_0.$$


---
## 2. Configuration Setup (Cartesian)

We use a thin $y$ dimension to keep the 3D case inexpensive while preserving Cartesian geometry.


In [ ]:
# Standard imports
import jax
import jax.numpy as jnp
import jax.lax as lax
import numpy as np
import matplotlib.pyplot as plt

# jax-frc imports
from jax_frc.configurations import FrozenFluxConfiguration
from jax_frc.solvers.explicit import RK4Solver
from jax_frc.validation.metrics import l2_error

# Notebook utilities
import sys
sys.path.insert(0, '.')
from _shared import plot_comparison, plot_error

# Plotting style
%matplotlib inline
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 11


In [ ]:
# === PHYSICS PARAMETERS ===
B_phi_0 = 1.0      # Uniform B_y magnitude [T]
v_r = 0.1          # Uniform v_x [m/s]
eta = 1e-8         # Very small resistivity (Rm >> 1)

# === GRID PARAMETERS (3D CARTESIAN) ===
nx, ny, nz = 64, 1, 64
r_min = 0.2        # x_min [m]
r_max = 1.0        # x_max [m]
z_extent = 0.5     # y,z in [-z_extent, z_extent]

config = FrozenFluxConfiguration(
    B_phi_0=B_phi_0,
    v_r=v_r,
    eta=eta,
    nx=nx,
    ny=ny,
    nz=nz,
    r_min=r_min,
    r_max=r_max,
    z_extent=z_extent,
)

runtime = config.default_runtime()
t_end = runtime['t_end']
dt = runtime['dt']

print(f"Grid: {nx} x {ny} x {nz}")
print(f"t_end = {t_end:.3e} s, dt = {dt:.3e} s")
print(f"Magnetic Reynolds number: Rm = {config.magnetic_reynolds_number():.2e}")


In [ ]:
# Build simulation components
geometry = config.build_geometry()
state0 = config.build_initial_state(geometry)
model = config.build_model()
solver = RK4Solver()

y_idx = geometry.ny // 2
z_idx = geometry.nz // 2
x = np.array(geometry.x_grid[:, y_idx, z_idx])
B_y0 = np.array(state0.B[:, y_idx, z_idx, 1])

fig, _ = plot_comparison(
    x,
    numerical=B_y0,
    analytic=np.ones_like(B_y0) * B_phi_0,
    xlabel="x [m]",
    ylabel="B_y [T]",
    title="Initial B_y vs analytic (constant)",
)
plt.show()


---
## 3. Run Simulation

We evolve the system with the solver under test. Since $\nabla \times (\mathbf{v} \times \mathbf{B}) = 0$ for uniform fields, the analytic solution is constant in time.


In [ ]:
n_steps = int(t_end / dt)
print(f"Running {n_steps} steps...")

@jax.jit
def scan_step(state, _):
    new_state = solver.step(state, dt, model, geometry)
    return new_state, new_state.B[:, y_idx, z_idx, 1]

final_state, B_y_history = lax.scan(scan_step, state0, None, length=n_steps)
B_y_final = np.array(final_state.B[:, y_idx, z_idx, 1])


In [ ]:
B_y_expected = np.ones_like(B_y_final) * B_phi_0
l2 = float(l2_error(jnp.asarray(B_y_final), jnp.asarray(B_y_expected)))
print(f"L2 error: {l2:.4g} (threshold: 0.01)")

fig, _ = plot_comparison(
    x,
    numerical=B_y_final,
    analytic=B_y_expected,
    xlabel="x [m]",
    ylabel="B_y [T]",
    title=f"B_y at y=0, z=0, t={t_end:.2e}s",
)
plt.show()

fig_err, _ = plot_error(
    x,
    numerical=B_y_final,
    analytic=B_y_expected,
    xlabel="x [m]",
    title="B_y Error (Simulation - Analytic)",
)
plt.show()


---
## Summary

This notebook demonstrated **Cartesian frozen-in flux** in the ideal-MHD limit:

1. **Physics**: The induction equation reduces to $\partial_t \mathbf{B} = \nabla \times (\mathbf{v} \times \mathbf{B})$ when $R_m \gg 1$.
2. **Analytic solution**: With uniform $\mathbf{v}$ and uniform $\mathbf{B}$, the field remains constant in time.
3. **Validation**: The solver preserves $B_y$ within the specified L2 tolerance (0.01).

### Try Next

- Increase `eta` to observe diffusion-driven decay ($R_m \sim 1$)
- Compare with the **magnetic diffusion** notebook for the opposite limit ($R_m \ll 1$)
